<a href="https://colab.research.google.com/github/1000century/ml/blob/main/%ED%86%A0%ED%81%AC%EB%82%98%EC%9D%B4%EC%A0%80_%EA%B2%B0%EA%B3%BC%EB%B9%84%EA%B5%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token='</s>',
    eos_token='</s>',
    unk_token='<unk>',
    pad_token='<pad>',
    mask_token='<mask>'
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


# 함수


In [ ]:
def visualize_sequence_predictions(text, pred, tokenizer, chunk_size=10):
    """
    입력 시퀀스와 예측을 비교하여 보여주는 함수
    각 토큰을 5글자 너비로 고정하고 초과시 두 줄로 표시
    토큰 개수가 다르더라도 모든 토큰을 표시
    각 토큰 위에 순번과 tokenizer의 token ID를 표시
    """
    # 텍스트를 토큰화
    tokens = tokenizer.tokenize(text)
    pred_tokens = tokenizer.tokenize(pred)

    # 토큰 ID 얻기
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    pred_token_ids = tokenizer.convert_tokens_to_ids(pred_tokens)

    html_style = """
    <style>
        .sequence-viewer {
            font-family: monospace;
            margin: 20px;
            line-height: 1.5;
        }
        .chunk {
            margin-bottom: 20px;
            padding: 10px;
            border: 1px solid #ddd;
            border-radius: 4px;
            background: #f8f9fa;
        }
        .index {
            color: #666;
            font-size: 0.9em;
            margin-bottom: 5px;
        }
        .token-wrapper {
            display: inline-flex;
            flex-direction: column;
            align-items: center;
            margin: 0 1px;
        }
        .token-index {
            font-size: 0.7em;
            color: #666;
            line-height: 1.1em;
        }
        .token-number {
            color: #1976d2;
        }
        .token-id {
            color: #7b1fa2;
        }
        .token {
            display: inline-block;
            width: 5em;  /* 5글자 너비로 고정 */
            height: 2.4em;  /* 2줄 높이 */
            padding: 2px 4px;
            border-radius: 3px;
            text-align: center;
            white-space: pre-wrap;
            word-break: break-all;
            overflow-wrap: break-word;
            vertical-align: middle;
            font-size: 0.9em;
            line-height: 1.2em;
        }
        .sequence-container {
            display: grid;
            grid-template-columns: auto 1fr;
            gap: 10px;
            margin-bottom: 10px;
        }
        .tokens-container {
            display: flex;
            flex-wrap: wrap;
            gap: 4px;
            align-items: flex-start;
        }
        .input {
            background-color: #e3f2fd;
            border: 1px solid #90caf9;
        }
        .prediction {
            background-color: #f3e5f5;
            border: 1px solid #ce93d8;
        }
        .match {
            color: #2e7d32;
        }
        .mismatch {
            color: #c62828;
        }
        .row-label {
            width: 50px;
            flex-shrink: 0;
            padding-top: 2px;
        }
        .full-text {
            margin-top: 10px;
            padding: 8px;
            background: #fff;
            border: 1px solid #e0e0e0;
            border-radius: 4px;
            font-family: 'Noto Sans KR', sans-serif;
            line-height: 1.6;
        }
        .text-label {
            font-weight: bold;
            color: #555;
            margin-right: 8px;
        }
        .extra-token {
            background-color: #fff3e0;
            border: 1px solid #ffe0b2;
        }
    </style>
    """

    html_content = f"{html_style}<div class='sequence-viewer'>"

    # 전체 시퀀스를 chunk_size 단위로 처리
    max_chunks = max(
        (len(tokens) + chunk_size - 1) // chunk_size,
        (len(pred_tokens) + chunk_size - 1) // chunk_size
    )

    for chunk_idx in range(max_chunks):
        start_idx = chunk_idx * chunk_size

        # 현재 청크의 토큰들 가져오기
        chunk_tokens = tokens[start_idx:start_idx + chunk_size] if start_idx < len(tokens) else []
        chunk_preds = pred_tokens[start_idx:start_idx + chunk_size] if start_idx < len(pred_tokens) else []
        chunk_token_ids = token_ids[start_idx:start_idx + chunk_size] if start_idx < len(token_ids) else []
        chunk_pred_token_ids = pred_token_ids[start_idx:start_idx + chunk_size] if start_idx < len(pred_token_ids) else []

        # 현재 청크의 토큰들로 문장 조각 만들기
        chunk_text = tokenizer.convert_tokens_to_string(chunk_tokens)
        chunk_pred_text = tokenizer.convert_tokens_to_string(chunk_preds) if chunk_preds else ""

        # 청크 시작
        html_content += f"""
        <div class='chunk'>
            <div class='index'>Tokens {start_idx + 1}-{min(start_idx + chunk_size, max(len(tokens), len(pred_tokens)))}</div>
        """

        # 입력 시퀀스 표시
        html_content += """
        <div class='sequence-container'>
            <div class='row-label'>입력:</div>
            <div class='tokens-container'>
        """
        if chunk_tokens:
            html_content += "".join([
                f"""<div class='token-wrapper'>
                    <div class='token-index'>
                        <span class='token-number'>{start_idx + i + 1}</span>
                        <span class='token-id'>[{token_id}]</span>
                    </div>
                    <span class='token input'>{t}</span>
                </div>"""
                for i, (t, token_id) in enumerate(zip(chunk_tokens, chunk_token_ids))
            ])
        html_content += "</div></div>"

        # 예측 시퀀스 표시
        html_content += """
        <div class='sequence-container'>
            <div class='row-label'>예측:</div>
            <div class='tokens-container'>
        """
        for j, (pred_token, pred_token_id) in enumerate(zip(chunk_preds, chunk_pred_token_ids)):
            # 입력 토큰이 있는 경우 매칭 여부 확인
            if j < len(chunk_tokens):
                match = pred_token == chunk_tokens[j]
                html_content += f"""<div class='token-wrapper'>
                    <div class='token-index'>
                        <span class='token-number'>{start_idx + j + 1}</span>
                        <span class='token-id'>[{pred_token_id}]</span>
                    </div>
                    <span class='token prediction {('match' if match else 'mismatch')}'>{pred_token}</span>
                </div>"""
            else:
                # 입력 토큰을 초과하는 예측 토큰은 별도 스타일로 표시
                html_content += f"""<div class='token-wrapper'>
                    <div class='token-index'>
                        <span class='token-number'>{start_idx + j + 1}</span>
                        <span class='token-id'>[{pred_token_id}]</span>
                    </div>
                    <span class='token prediction extra-token'>{pred_token}</span>
                </div>"""
        html_content += "</div></div>"

        # 현재 청크의 원본 텍스트와 예측 텍스트 표시
        html_content += f"""
        <div class='full-text'>
            <div><span class='text-label'>원문:</span>{chunk_text}</div>
            <div><span class='text-label'>예측:</span>{chunk_pred_text}</div>
        </div>
        """

        html_content += "</div>"

    html_content += "</div>"

    from IPython.display import HTML
    return HTML(html_content)


# 실행

In [ ]:
pred = '직무: 공공 경력상태: 신입직 질문: 지원 후에 지원 모습의 되고 싶으요 왜변: 저 후 저 하면서 하고 사람이 그리고 일을 열심히 하는 싶적으로 일하는 사람이 되고 싶습니다. 어들 하는 사람은 열정하지 못하는 일을 사람이 그 더 성과를 성과를 성과를와 성과를적얄 이끌어 수 있다고 저는 일을적으로 열정 사람이 사람이 되고겠 노력하겠습니다. 또한                                                                   저   저           저    저    저    저    저   저     저  저 저   저 저  저     저  저 저  저 저 저 저 저 저 저 저  저 저  저  저 저  저 저 저 저  저  저 저 저 저 저 저 저  저 저 저 저 저 저  저 저 저 저 저 저 저 저 저 저 저 저 저 저 저 저 저 저 저 저 저'

label = '직무: 연구개발 경력상태: 경력직 질문: 입사하면 어떤 사람이 되고 싶나요. 답변: 입사하면 일을 열심히 하는 사람보다는 일을 효율적으로 하고 열정적으로 하는 사람이 되고 싶습니다. 흔히 열심히 하는 것보다 잘하고 효율적으로 하는 것은 보다 많은 업무적 성과나 업무 로드를 줄일 수 있으므로 저는 열정적이고 잘하는 효율적인 사람이 되 되도록 노력하겠습니다.'

In [ ]:

# 사용 예시:
visualize_sequence_predictions(label, pred, tokenizer, chunk_size=18)

# 모델 실행 함수 정의

In [ ]:
tok_style = """
    <style>
        .prediction-viewer {
            font-family: monospace;
            margin: 20px;
            line-height: 1.5;
            font-size: 0.95em;
        }
        .chunk {
            margin-bottom: 40px;
            padding: 20px;
            border: 1px solid #ddd;
            border-radius: 8px;
            background: #f8f9fa;
            overflow-x: auto;
        }
        .chunk-header {
            font-weight: bold;
            margin-bottom: 15px;
            padding: 5px 10px;
            background: #e3f2fd;
            border-radius: 4px;
        }
        .chunk-sentences {
            margin: 10px 0;
            padding: 10px;
            background: #fff;
            border: 1px solid #e0e0e0;
            border-radius: 4px;
            font-family: sans-serif;
            line-height: 1.6;
        }
        .sentence-label {
            font-weight: bold;
            color: #1976d2;
            margin-right: 8px;
        }
        .tokens-row {
            display: flex;
            gap: 8px;
            margin: 15px 0;
            min-width: max-content;
        }
        .token-column {
            flex: 0 0 70px;
            display: flex;
            flex-direction: column;
        }
        .token-info {
            padding: 4px;
            background: #f3e5f5;
            border-radius: 4px;
            text-align: center;
            margin-bottom: 4px;
            border: 1px solid #e0e0e0;
        }
        .token-text {
            font-weight: bold;
            font-size: 0.9em;
            word-break: break-all;
            white-space: normal;
        }
        .token-id {
            color: #7b1fa2;
            font-size: 0.8em;
        }
        .predictions-list {
            display: flex;
            flex-direction: column;
            gap: 2px;
        }
        .prediction-item {
            padding: 2px;
            background: white;
            border: 1px solid #e0e0e0;
            border-radius: 4px;
            font-size: 0.85em;
            text-align: center;
        }
        .prediction-token {
            word-break: break-all;
            white-space: normal;
            margin-bottom: 1px;
        }
        .prediction-prob {
            color: #1976d2;
            font-size: 0.95em;
            margin-bottom: 1px;
        }
        .prediction-id {
            color: #7b1fa2;
            font-size: 0.95em;
        }
        .selected {
            background: #f1f8e9;
            border: 2px solid #4caf50;
        }
    </style>
    """

In [ ]:
import torch
from IPython.display import HTML

def visualize_token_predictions(input_text, model, tokenizer, device='cuda', top_k=5, chunk_size=18):
    """
    입력 텍스트의 각 위치별 실제 토큰과 모델이 예측한 top-k 토큰들을 시각화
    """
    html_style = tok_style
    input_text = '</s>' + input_text + '</s>'

    # 입력 텍스트 토큰화
    inputs = tokenizer(input_text, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)

    # 한번에 전체 시퀀스에 대한 예측 받기
    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits[0]  # [seq_len, vocab_size]
        probs = torch.softmax(logits, dim=-1)
        top_probs, top_indices = torch.topk(probs, top_k, dim=-1)  # [seq_len, top_k]

    # 전체 문장에 대한 실제 텍스트와 예측 텍스트
    actual_full_text = tokenizer.decode(input_ids[0])
    predicted_full_ids = top_indices[:, 0]  # 각 위치에서 가장 높은 확률의 토큰
    predicted_full_text = tokenizer.decode(predicted_full_ids)

    html_content = f"{html_style}<div class='prediction-viewer'>"

    # 전체 문장 표시 추가
    html_content += f"""
    <div class='full-text-section'>
        <div class='full-text-header'>전체 문장</div>
        <div class='full-text-content'>
            <div><span class='sentence-label'>원본:</span>{actual_full_text}</div>
            <div><span class='sentence-label'>예측:</span>{predicted_full_text}</div>
        </div>
    </div>
    """

    # chunk_size 단위로 나눠서 표시
    for chunk_start in range(0, input_ids.shape[1], chunk_size):
        chunk_end = min(chunk_start + chunk_size, input_ids.shape[1])
        chunk_tokens = input_ids[0, chunk_start:chunk_end]

        html_content += f"""
        <div class='chunk'>
            <div class='chunk-header'>Tokens {chunk_start + 1}-{chunk_end}</div>
            <div class='tokens-row'>
        """

        # 각 토큰 위치별로 실제 토큰과 예측 표시
        for i, token_id in enumerate(chunk_tokens):
            pos = chunk_start + i
            current_token = tokenizer.convert_ids_to_tokens([token_id])[0]

            html_content += f"""
            <div class='token-column'>
                <div class='token-info'>
                    <div class='token-text'>{current_token}</div>
                    <div class='token-id'>[{token_id.item()}]</div>
                </div>
                <div class='predictions-list'>
            """

            # top-k 예측 결과 표시
            for j in range(top_k):
                pred_id = top_indices[pos, j]
                pred_token = tokenizer.convert_ids_to_tokens([pred_id])[0]
                pred_prob = top_probs[pos, j]

                is_actual = False
                if pos < len(input_ids[0]) - 1:
                    next_token_id = input_ids[0, pos + 1]
                    is_actual = pred_id == next_token_id

                html_content += f"""
                    <div class='prediction-item{' selected' if is_actual else ''}'>
                        <div class='prediction-token'>{pred_token}</div>
                        <div class='prediction-prob'>{pred_prob.item():.4f}</div>
                        <div class='prediction-id'>[{pred_id.item()}]</div>
                    </div>
                """

            html_content += "</div></div>"

        html_content += "</div>"  # tokens-row 종료

        # 현재 청크의 실제 텍스트와 가장 높은 확률의 예측으로 만든 텍스트 표시
        actual_text = tokenizer.decode(chunk_tokens)
        predicted_ids = top_indices[chunk_start:chunk_end, 0]
        predicted_text = tokenizer.decode(predicted_ids)

        html_content += f"""
            <div class='chunk-sentences'>
                <div><span class='sentence-label'>원본:</span>{actual_text}</div>
                <div><span class='sentence-label'>예측:</span>{predicted_text}</div>
            </div>
        </div>
        """

    html_content += "</div>"
    return HTML(html_content)

# 트레이너 모델

## 트레이너 모델 불러오기

In [ ]:
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token='</s>',
    eos_token='</s>',
    unk_token='<unk>',
    pad_token='<pad>',
    mask_token='<mask>'
)

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [ ]:
import json
import os

# config.json 파일 생성
config_dict = {
    "_name_or_path": "skt/kogpt2-base-v2",
    "_num_labels": 1,
    "activation_function": "gelu_new",
    "architectures": ["GPT2LMHeadModel"],
    "attn_pdrop": 0.1,
    "author": "Heewon Jeon(madjakarta@gmail.com)",
    "bos_token_id": 0,
    "created_date": "2021-04-28",
    "embd_pdrop": 0.1,
    "eos_token_id": 1,
    "gradient_checkpointing": False,
    "id2label": {"0": "LABEL_0"},
    "initializer_range": 0.02,
    "label2id": {"LABEL_0": 0},
    "layer_norm_epsilon": 0.00001,
    "license": "CC-BY-NC-SA 4.0",
    "model_type": "gpt2",
    "n_ctx": 1024,
    "n_embd": 768,
    "n_head": 12,
    "n_inner": None,
    "n_layer": 12,
    "n_positions": 1024,
    "pad_token_id": 3,
    "reorder_and_upcast_attn": False,
    "resid_pdrop": 0.1,
    "scale_attn_by_inverse_layer_idx": False,
    "scale_attn_weights": True,
    "summary_activation": None,
    "summary_first_dropout": 0.1,
    "summary_proj_to_labels": True,
    "summary_type": "cls_index",
    "summary_use_proj": True,
    "task_specific_params": {
        "text-generation": {
            "do_sample": True,
            "max_length": 50
        }
    },
    "torch_dtype": "float32",
    "transformers_version": "4.45.1",
    "use_cache": True,
    "vocab_size": 51200
}

# 디렉토리 생성
os.makedirs("results/best_model", exist_ok=True)

# config.json 파일 저장
with open("results/best_model/config.json", "w", encoding="utf-8") as f:
    json.dump(config_dict, f, indent=2)

# safetensors 파일을 results/best_model/model.safetensors로 이동하거나 복사
# !cp /content/model.safetensors results/best_model/model.safetensors

# 이제 모델을 로드할 수 있습니다
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("results/best_model")

In [ ]:
import torch

# 모델이 DataParallel이나 DistributedDataParallel로 래핑되어 있는지 확인
if isinstance(model, torch.nn.DataParallel) or isinstance(model, torch.nn.parallel.DistributedDataParallel):
    model = model.module  # 원래 모델 추출

# 모델의 모든 파라미터를 단일 GPU로 이동
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 모델을 평가 모드로 설정
model.eval()

print("Model moved to:", device)
print("\nModel architecture:")
print(model)

Model moved to: cuda:0

Model architecture:
GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)


## 트레이너 모델 예시보기

In [ ]:

# 사용 예시:
text = '직무: 연구개발 경력상태: 경력직 질문: 입사하면 어떤 사람이 되고 싶나요. 답변: 입사하면 일을 열심히 하는 사람보다는 일을 효율적으로 하고 열정적으로 하는 사람이 되고 싶습니다. 흔히 열심히 하는 것보다 잘하고 효율적으로 하는 것은 보다 많은 업무적 성과나 업무 로드를 줄일 수 있으므로 저는 열정적이고 잘하는 효율적인 사람이 되 되도록 노력하겠습니다.'
visualize_token_predictions(text, model, tokenizer, device=device, top_k=10, chunk_size=14)

In [ ]:

# 사용 예시:
text = '직무: 생산관리 질문: 요즘 공부를 하고 있는 것이 있는지 있으면 그걸 왜 공부하고 있는지 그리고 지금까지 하면서 무엇을 배웠는지도 설명해 주시기 바랍니다 답변: 요즘 저는 언어 공부를 하고 있습니다. 특히 중국어를 공부를 하고 있는데 중국어는 제가 고등학교 때 처음으로 학교에서 제 이 외국어로 배웠던 경험이 있습니다. 학교 선생님께서는 앞으로의 중국 시장은 무한이 넓고 무엇이든지 할 수 있어 라고 가르쳐주셨기 때문에 그때만 해도 그냥 중문과를 가서 중국어를 했었습니다. 그런데 문법만 공부했던 것이라 잘 되지 않았는데 이것을 다시 중국어를 공부를 하고 중국어 회화를 하고 있습니다. 중국 시장을 가서 바이어를 만나고 그 바이어에게 그 물건을 팔면 그 내 물건은 정말 몇 십억 인구에게 전해지는 아주 좋은 획기적인 그런 일이 생길 수 있습니다. 제일 먼저 그걸 해야 해야 되는 이유 중의 하나가 바로 언어 공부입니다. 언어를 해서 상대방 바이어들에게 그것이 어떤 건지를 잘 가르켜 주고 그리고 그것이 잘 할 수 있도록 해주는 것이 언어이기 때문에 우리는 확실하게 언어를 해야 된다고 생각하고 있습니다. 뭐든지 열심히 하면 잘 될 수 있고 그리고 회화를 하려면 중국 드라마도 많이 보고 있고 중국 회화도 계속 이어폰을 끼고 들어서 내 입에서 계속 중국어가 나올 수 있게끔 성조가 잘 익혀지게끔 하는 것을 하고 있습니다. '

visualize_token_predictions(text, model, tokenizer, device=device, top_k=10, chunk_size=14)

# Pytorch 버젼 모델

## Pytorch 버젼 모델 불러오기

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2TokenizerFast


# Load the pretrained model
py_model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
model.eval()

# Move model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
py_model.to(device)
# 2. Load custom weights and generate response again
py_model.load_state_dict(torch.load('./best_model.pt', map_location=torch.device('cpu')))
py_model.eval()

<ipython-input-46-488dd828e966>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  py_model.load_state_dict(torch.load('./best_model.pt', map_location=torch.device('cpu')))


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

## Pytorch 버젼 모델 예시

In [ ]:
text = '직무: 생산관리 질문: 요즘 공부를 하고 있는 것이 있는지 있으면 그걸 왜 공부하고 있는지 그리고 지금까지 하면서 무엇을 배웠는지도 설명해 주시기 바랍니다 답변: 요즘 저는 언어 공부를 하고 있습니다. 특히 중국어를 공부를 하고 있는데 중국어는 제가 고등학교 때 처음으로 학교에서 제 이 외국어로 배웠던 경험이 있습니다. 학교 선생님께서는 앞으로의 중국 시장은 무한이 넓고 무엇이든지 할 수 있어 라고 가르쳐주셨기 때문에 그때만 해도 그냥 중문과를 가서 중국어를 했었습니다. 그런데 문법만 공부했던 것이라 잘 되지 않았는데 이것을 다시 중국어를 공부를 하고 중국어 회화를 하고 있습니다. 중국 시장을 가서 바이어를 만나고 그 바이어에게 그 물건을 팔면 그 내 물건은 정말 몇 십억 인구에게 전해지는 아주 좋은 획기적인 그런 일이 생길 수 있습니다. 제일 먼저 그걸 해야 해야 되는 이유 중의 하나가 바로 언어 공부입니다. 언어를 해서 상대방 바이어들에게 그것이 어떤 건지를 잘 가르켜 주고 그리고 그것이 잘 할 수 있도록 해주는 것이 언어이기 때문에 우리는 확실하게 언어를 해야 된다고 생각하고 있습니다. 뭐든지 열심히 하면 잘 될 수 있고 그리고 회화를 하려면 중국 드라마도 많이 보고 있고 중국 회화도 계속 이어폰을 끼고 들어서 내 입에서 계속 중국어가 나올 수 있게끔 성조가 잘 익혀지게끔 하는 것을 하고 있습니다. '


visualize_token_predictions(text, py_model, tokenizer, device=device, chunk_size=14,top_k=10)